<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/кейсы%20по%20анализу%20данных/Штаты.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

In [20]:
import pandas as pd
import numpy as np

Операции слияния и соединения чаще всего оказываются нужны при объединении данных из различных источников. Рассмотрим пример с определенными данными по штатам США и их населению.

Посмотрим на эти наборы данных с помощью функции read_csv() библиотеки Pandas:

In [25]:
pop = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv')
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [26]:
areas = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv')
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [27]:
abbrevs = pd.read_csv('https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv')
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


Допустим, нам нужно на основе этой информации отсортировать штаты и территорию США по плотности населения в 2010 году. Информации для этого у нас достаточно, но для достижения цели придется объединить наборы данных.
Начнем со слияния «многие-ко-многим», которое позволит получить полное имя штата в объекте DataFrame для населения. Выполнить слияние нужно на основе столбца state/region объекта pop и столбца abbreviation объекта abbrevs . Мы воспользуемся опцией how='outer' , чтобы гарантировать, что не упустим никаких данных из-за несовпадения меток.

In [28]:
merged = pd.merge(pop, abbrevs, how='outer', left_on='state/region', right_on='abbreviation')
merged.head()

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL


Удаляем дублирующуюся информацию:

In [29]:
merged = merged.drop('abbreviation', axis=1) 
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Следует проверить, не было ли каких-то несовпадений. Сделать это можно путем поиска строк с пустыми значениями:

In [30]:
merged.isnull()

,state/region,ages,year,population,state
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
2539,False,False,False,False,True
2540,False,False,False,False,True
2541,False,False,False,False,True
2542,False,False,False,False,True


In [31]:
merged.isnull().any() 

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Часть информации о населении отсутствует, выясним, какая именно:

In [32]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


Похоже, что источник пустых значений по населению — Пуэрто-Рико, до 2000 года.
Вероятно, это произошло из-за того, что необходимых данных не было в первоисточнике.
Мы видим, что некоторые из новых значений столбца state тоже пусты, а значит, в ключе объекта abbrevs отсутствовали соответствующие записи! Выясним, для каких территорий отсутствуют соответствующие значения:

In [33]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

Все понятно: наши данные по населению включают записи для Пуэрто-Рико (PR) и США в целом (USA), отсутствующие в ключе аббревиатур штатов. Исправим это, вставив соответствующие записи:

In [34]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

В столбце state больше нет пустых значений. Готово!

Теперь можно слить результат с данными по площади штатов с помощью аналогичной процедуры. После изучения имеющихся результатов становится понятно, что нужно выполнить соединение по столбцу state в обоих объектах:

In [35]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Выполним снова проверку на пустые значения, чтобы узнать, были ли какие-то несовпадения:

In [36]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

В столбце area имеются пустые значения. Посмотрим, какие территории не были учтены:

In [37]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

Видим, что наш DataFrame -объект areas не содержит площадь США в целом.
Мы могли бы вставить соответствующее значение (например, воспользовавшись суммой площадей всех штатов), но в данном случае мы просто удалим пустые значения, поскольку плотность населения США в целом нас сейчас не
интересует:

In [38]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Теперь у нас есть все необходимые нам данные. Чтобы ответить на интересующий вопрос, сначала выберем часть данных, соответствующих 2010 году и всему населению.

In [39]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Теперь вычислим плотность населения и выведем данные в соответствующем порядке. Начнем с переиндексации наших данных по штату, после чего вычислим результат:

In [40]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [41]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

Результат — список штатов США плюс Вашингтон (округ Колумбия) и Пуэрто-Рико, упорядоченный по плотности населения в 2010 году, в жителях на квадратную милю. Как видим, самая густонаселенная территория в этом наборе данных — Вашингтон (округ Колумбия); среди штатов же самый густонаселенный — Нью-Джерси.
Можно также вывести окончание списка:

In [42]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

Как видим, штатом с наименьшей плотностью населения, причем с большим отрывом от остальных, оказалась Аляска, насчитывающая в среднем одного жителя на квадратную милю.

Подобное громоздкое слияние данных — распространенная задача при ответе на вопросы, связанные с реальными источниками данных. Надеюсь, что этот пример дал вам представление, какими способами можно комбинировать вышеописанные инструменты, чтобы почерпнуть полезную информацию из данных!